In [1]:
#! pip install https://repository.tomtomgroup.com/repository/nexus-maps-analytics-utils/packages/maps-analytics-utils/0.0.1/maps_analytics_utils-0.0.1-py3-none-any.whl
! pip install /workspace/main_folder/wheels/*
! pip install pycountry
! pip install country_list
! pip install country_converter
! pip install regex
! pip install matplotlib
! pip install pyarrow
! pip install shapely
! pip install geopandas

In [1]:
import pandas as pd
import numpy as np
import pycountry
from maps_analytics_utils.connections import adx, connections_utils
from country_list import countries_for_language, available_languages
import typing
import country_converter
import json
import re
import matplotlib.pyplot as plt
import regex
from datetime import date
from pyspark.sql.functions import DataFrame
import datetime
import auxiliary_functions.generating_ADX_sample

## Setting up the country to use:

In [3]:
today = datetime.datetime.today().strftime('%d-%m-%Y')
countries = ['US']
endpoint_list = None  ## If you are using only 1 endpoint you should use a comma at the end: ('search 2 search', )
table = 'search_logs_insights'
only_full_addresses = False
sample = 500000
ago = '720'
exclude_endpoint = ('search 2 poiSearch', 'search 2 nearbySearch')
developer_emails_list = 'maps.analytics.metrics@groups.tomtom.com'
deduplicate = True

***Below there's a description of Libpostal's address components***

- **house:** venue name e.g. "Brooklyn Academy of Music", and building names e.g. "Empire State Building"
- **near:** phrases like "in", "near", etc. used after a category phrase to help with parsing queries like "restaurants in Brooklyn"
- **house_number:** usually refers to the external (street-facing) building number. In some countries this may be a compount, hyphenated number which also includes an apartment number, or a block number (a la Japan), but libpostal will just call it the house_number for simplicity.
- **road:** street name(s)
- **unit:** an apartment, unit, office, lot, or other secondary unit designator
- **level:** expressions indicating a floor number e.g. "3rd Floor", "Ground Floor", etc.
- **staircase:** numbered/lettered staircase
- **entrance:** numbered/lettered entrance
- **po_box:** post office box: typically found in non-physical (mail-only) addresses
- **postcode:** postal codes used for mail sorting
- **suburb:** usually an unofficial neighborhood name like "Harlem", "South Bronx", or "Crown Heights"
- **city_district:** these are usually boroughs or districts within a city that serve some official purpose e.g. "Brooklyn" or "Hackney" or "Bratislava IV"
- **city:** any human settlement including cities, towns, villages, hamlets, localities, etc.
- **island:** named islands e.g. "Maui"
- **state_district:** usually a second-level administrative division or county.
- **state:** a first-level administrative division. Scotland, Northern Ireland, Wales, and England in the UK are mapped to "state" as well (convention used in OSM, GeoPlanet, etc.)
- **country_region:** informal subdivision of a country without any political status
- **country:** sovereign nations and their dependent territories, anything with an ISO-3166 code.
- **world_region:** currently only used for appending “West Indies” after the country name, a pattern frequently used in the English-speaking Caribbean e.g. “Jamaica, West Indies”

## Complete addresses

Include only complete addresses in the responses. This is the safest bet, as you get only what the API deems as a complete address, and so you get better responses (more complete). The problem is that you will get much less variability in your data. So if you want geometries, incomplete addresses, searches up to road level, etc., this is not your function, as it will be problematic.

In [5]:
if only_full_addresses:
    responses_dict_requests = auxiliary_functions.generating_ADX_sample.address_components_sample_generator(
        country_list=countries, 
        endpoint_list=endpoint_list,
        table=table,
        sample=sample,
        developer_emails_list=developer_emails_list,
        populated_fields=None,
        only_full_addresses=only_full_addresses,
        ago=ago,
        check_query=False,
        deduplicate=deduplicate, 
        exclude_endpoint_list=exclude_endpoint,
        filter_results=None
    )

## All addresses (non-complete included)

Use this function to if you are interested in getting the full array of responses (not only complete addresses) use this function instead. The function is the same, it simply changes a parameter so that you can get the geometries, roads, APTs, etc., within the same query.

In [4]:
if not only_full_addresses:
    responses_dict_requests = auxiliary_functions.generating_ADX_sample.address_components_sample_generator(
        country_list=countries, 
        endpoint_list=endpoint_list,
        table=table,
        sample=sample,
        developer_emails_list=developer_emails_list,
        populated_fields=None,
        only_full_addresses=only_full_addresses,
        ago=ago,
        check_query=False,
        deduplicate=deduplicate,
        exclude_endpoint_list=exclude_endpoint
    )

KustoMultiApiError: Query execution has exceeded the allowed limits (80DA0001): 'summarize' operator has exceeded the memory budget (5368709120) during evaluation. Results may be incorrect or incomplete (E_RUNAWAY_QUERY; see https://aka.ms/kustoquerylimits)..

## Saving the sample

In [78]:
if only_full_addresses:
    suffix = ''
else:
    suffix = 'in'

(
    auxiliary_functions.generating_ADX_sample.general_dbfs_save_function_dict_of_countries(
        responses_dict_requests, 'data', 'parquet', f'{suffix}complete_responses_{today}', sep=';'
    )
)

US is done in data/complete_responses_10-11-2022_US!!
